In [ ]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!cp kaggle.json ~/.kaggle/
!kaggle datasets list

ref                                                           title                                                size  lastUpdated          downloadCount  
------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
allen-institute-for-ai/CORD-19-research-challenge             COVID-19 Open Research Dataset Challenge (CORD-19)    3GB  2020-07-06 18:49:31          87234  
roche-data-science-coalition/uncover                          UNCOVER COVID-19 Challenge                          179MB  2020-05-21 18:57:53          15261  
oleksastepaniuk/prozorro-public-procurement-dataset           ProZorro. Ukrainian public procurement dataset      448MB  2020-04-15 17:09:37             32  
nareshbhat/health-care-data-set-on-heart-attack-possibility   Health care: Data set on Heart attack possibility     3KB  2020-06-25 07:11:26            762  
imdevskp/cholera-dataset                            

In [ ]:
# !kaggle competitions download -c house-prices-advanced-regression-techniques
!kaggle competitions download -c open-shopee-code-league-logistic
!ls

  0% 0.00/9.01k [00:00<?, ?B/s]
100% 9.01k/9.01k [00:00<00:00, 8.52MB/s]
 99% 378M/381M [00:06<00:00, 75.3MB/s]
100% 381M/381M [00:06<00:00, 58.9MB/s]
delivery_orders_march.csv.zip  kaggle.json  sample_data  SLA_matrix.xlsx


In [ ]:
# import specific libraries
# load CSVs
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
SLA_matrix = pd.read_excel('SLA_matrix.xlsx')
delivery_orders_march = pd.read_csv('delivery_orders_march.csv.zip')

In [ ]:
import time
import datetime
def date(input):
    date_time_str = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(input))
    final = date_time_str.split(' ')[0]
    date_time_obj = datetime.datetime.strptime(final, '%Y-%m-%d')
    return date_time_obj
def day_diff(att, pic):
    attempt = date(att)
    pick = date(pic)
    days = attempt - pick
    return days.days
def str_date(input):
    date_time_str = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(input))
    final = date_time_str.split(' ')[0]
    return final
import numpy as np
def business_days(inn,out):
    days = np.busday_count(inn,out,weekmask=[1,1,1,1,1,1,0],holidays=['2020-03-08','2020-03-25','2020-03-30','2020-03-31'])
    return int(days)

In [ ]:
def location(input):
    info = input[-15:]
    if 'Metro Manila' in info or 'metro manila' in info or 'manila' or 'Manila' in info:
        return 'Metro Manila'
    elif 'Luzon' in info or 'luzon' in info:
        return 'Luzon'
    elif 'Visayas' in info or 'visayas' in info:
        return 'Visayas'
    elif 'Mindanao' in info or 'mindanao' in info:
        return 'Mindanao'
    else:
        return 'inconclusive'

In [ ]:
delivery_orders_march['buy_city'] = delivery_orders_march.buyeraddress.apply (lambda row: location(row))
delivery_orders_march['sell_city'] = delivery_orders_march.selleraddress.apply (lambda row: location(row))

In [ ]:
delivery_orders_march['2nd_deliver_attempt'] = delivery_orders_march['2nd_deliver_attempt'].replace(np.nan, 0, regex=True)

In [ ]:
delivery_orders_march['str_pick'] = delivery_orders_march['pick'].apply (lambda row: str_date(row))
delivery_orders_march['str_1st_deliver_attempt'] = delivery_orders_march['1st_deliver_attempt'].apply (lambda row: str_date(row))
delivery_orders_march['str_2nd_deliver_attempt'] = delivery_orders_march['2nd_deliver_attempt'].apply (lambda row: str_date(row))

In [ ]:
delivery_orders_march['SLA_1'] = np.busday_count(delivery_orders_march['str_pick'],
                                     delivery_orders_march['str_1st_deliver_attempt'],
                                     weekmask=[1,1,1,1,1,1,0],
                                     holidays=['2020-03-08','2020-03-25','2020-03-30','2020-03-31'])

In [ ]:
delivery_orders_march['SLA_2'] = np.busday_count(delivery_orders_march['str_1st_deliver_attempt'],
                                     delivery_orders_march['str_2nd_deliver_attempt'],
                                     weekmask=[1,1,1,1,1,1,0],
                                     holidays=['2020-03-08','2020-03-25','2020-03-30','2020-03-31'])

In [ ]:
slas = pd.DataFrame({'Origin': ['Metro Manila', 'Luzon', 'Visayas', 'Mindanao'],
                     'Metro Manila': [3, 5, 7, 7],
                     'Luzon': [5, 5, 7, 7],
                     'Visayas': [7, 7, 7, 7],
                     'Mindanao': [7, 7, 7, 7]})

In [ ]:
def sla(row):
    if row == 'Metro ManilaMetro Manila':
        return 3
    if row == 'Metro ManilaLuzon':
        return 5
    if row == 'LuzonMetro Manila':
        return 5
    if row == 'LuzonLuzon':
        return 5
    else:
        return 7

In [ ]:
delivery_orders_march['combined'] = delivery_orders_march['buy_city'] + delivery_orders_march['sell_city']

In [ ]:
delivery_orders_march['allowed_sla'] = delivery_orders_march['combined'].apply(sla)

In [ ]:
delivery_orders_march['SLA2'] = delivery_orders_march.SLA_2.apply(changeneg)

In [ ]:
def changeneg(inp):
    if inp<0:
        return 0
    else:
        return inp

In [ ]:
delivery_orders_march.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,buy_city,sell_city,str_pick,str_1st_deliver_attempt,str_2nd_deliver_attempt,SLA_1,SLA_2,combined,allowed_sla,SLA2
0,2215676524,1583138397,1.583385e+09,0.000000e+00,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,Metro Manila,Metro Manila,2020-03-02,2020-03-05,1970-01-01,3,-15708,Metro ManilaMetro Manila,3,0
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",Metro Manila,Metro Manila,2020-03-04,2020-03-06,2020-03-10,2,3,Metro ManilaMetro Manila,3,3
2,2220979489,1583306434,1.583460e+09,0.000000e+00,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",Metro Manila,Metro Manila,2020-03-04,2020-03-06,1970-01-01,2,-15709,Metro ManilaMetro Manila,3,0
3,2221066352,1583419016,1.583556e+09,0.000000e+00,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",Metro Manila,Metro Manila,2020-03-05,2020-03-07,1970-01-01,2,-15710,Metro ManilaMetro Manila,3,0
4,2222478803,1583318305,1.583480e+09,0.000000e+00,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,Luzon,Metro Manila,2020-03-04,2020-03-06,1970-01-01,2,-15709,LuzonMetro Manila,5,0


In [ ]:
def check_late(row):
    if row.SLA_1 <= row.allowed_sla and row.SLA2 <= 3 and row.SLA2 != 0:
        return 0
    else:
        return 1

In [ ]:
delivery_orders_march['is_late'] = delivery_orders_march.apply(check_late, axis=1)

In [ ]:
forsub = delivery_orders_march[['orderid', 'is_late']].copy()

In [ ]:
forsub.to_csv('submission.csv', index=False)

In [ ]:
forsub

In [ ]:
!kaggle competitions submit -c open-shopee-code-league-logistic -f submission.csv -m "Message"